In [12]:
import os
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow.compute import kurtosis
import numpy as np
import librosa
import duckdb

from concurrent.futures import ThreadPoolExecutor

In [3]:
DATA_DIR = "../include/data/"

In [5]:
# # cloud
# SILVER_FOLDER_NAME = "sgppipelinesa-silver"
# SUB_FOLDER_NAME = "stage-01"
# SILVER_DATA_DIR = os.path.join(URL.format(FOLDER_NAME=SILVER_FOLDER_NAME), SUB_FOLDER_NAME)
# SILVER_DATA_DIR

# local
SILVER_FOLDER_NAME = "silver/"
SUB_FOLDER_NAME = "stage-01/"
SILVER_DATA_DIR = os.path.join(DATA_DIR, os.path.join(SILVER_FOLDER_NAME, SUB_FOLDER_NAME))
SILVER_DATA_DIR

'../include/data/silver/stage-01/'

In [11]:
subject_table = pq.read_table(os.path.join(SILVER_DATA_DIR, "_caustic_-20170306-smy_signals.parquet"))
subject_table

pyarrow.Table
signals: float
subjectId: string
rowId: int32
----
signals: [[0.0009460449,0.0008239746,0.00076293945,0.00064086914,0.00048828125,...,-0.077941895,-0.08255005,-0.0909729,-0.098846436,-0.10534668],[-0.10913086,-0.11035156,-0.10748291,-0.10131836,-0.08596802,...,-0.0005187988,0.001739502,-0.00030517578,0.00061035156,0.003692627],...,[-0.021118164,-0.0065612793,0.0034179688,0.004486084,-0.011230469,...,0.026397705,0.004425049,-0.0184021,-0.04437256,-0.06384277],[-0.067718506,-0.056793213,-0.04071045,-0.023620605,0.0061950684,...,0.008422852,0.009674072,0.010498047,0.011352539,0.012268066]]
subjectId: [["_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy",...,"_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy"],["_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy","_caustic_-20170306-smy

In [14]:
# connect to an in-memory database
conn = duckdb.connect()

In [32]:
conn.sql("""
    SELECT COUNT(*) FROM subject_table
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       553472 │
└──────────────┘

In [39]:
conn.sql("""
    SELECT
        signals, 
        subjectId, 
        rowId,
        sum(rowId) OVER(PARTITION BY subjectId ORDER BY rowId ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING) AS freq_kurt
    FROM subject_table
    ORDER BY rowId
""")

┌────────────────┬────────────────────────┬───────┬───────────┐
│    signals     │       subjectId        │ rowId │ freq_kurt │
│     float      │        varchar         │ int32 │  int128   │
├────────────────┼────────────────────────┼───────┼───────────┤
│   0.0009460449 │ _caustic_-20170306-smy │     0 │         1 │
│   0.0008239746 │ _caustic_-20170306-smy │     1 │         3 │
│  0.00076293945 │ _caustic_-20170306-smy │     2 │         5 │
│  0.00064086914 │ _caustic_-20170306-smy │     3 │         7 │
│  0.00048828125 │ _caustic_-20170306-smy │     4 │         9 │
│  0.00045776367 │ _caustic_-20170306-smy │     5 │        11 │
│   0.0005187988 │ _caustic_-20170306-smy │     6 │        13 │
│   0.0004272461 │ _caustic_-20170306-smy │     7 │        15 │
│  0.00030517578 │ _caustic_-20170306-smy │     8 │        17 │
│  0.00039672852 │ _caustic_-20170306-smy │     9 │        19 │
│        ·       │           ·            │     · │         · │
│        ·       │           ·          

In [34]:
conn.sql("""
    SELECT COUNT(*) FROM subject_table
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       553472 │
└──────────────┘